# Imports

In [50]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from utilsforecast.plotting import plot_series
from datetime import datetime as dt

In [2]:
pd.plotting.register_matplotlib_converters()
plt.rc("figure", figsize=(10, 8))
plt.rc("font", size=10)

In [51]:
df = pd.read_csv("/workspaces/SE-Europe-Data_Challenge/data/raw/2022_to_2023/gen_DE_B01.csv")

In [52]:
df.drop(['EndTime', 'UnitName', 'AreaID', 'PsrType'], axis=1, inplace=True)
df.rename(columns={'StartTime': 'start_time', 'quantity': 'energy_generated'}, inplace=True)

In [53]:
df.sort_values(['start_time'], inplace=True)

In [54]:
type(df['start_time'])

pandas.core.series.Series

In [55]:
df.head()

,start_time,energy_generated
0,2021-12-31T23:45+00:00Z,4333
1,2022-01-01T00:00+00:00Z,4325
2,2022-01-01T00:15+00:00Z,4319
3,2022-01-01T00:30+00:00Z,4323
4,2022-01-01T00:45+00:00Z,4328


In [45]:

d1 = dt.strptime("2021-12-31T23:45+00:00Z","%Y-%m-%dT%H:%M%zZ")

In [46]:
d1

datetime.datetime(2021, 12, 31, 23, 45, tzinfo=datetime.timezone.utc)

In [47]:
new_format = "%Y-%m-%d %H:%M:%S"

In [48]:
d1.strftime(new_format)

'2021-12-31 23:45:00'

In [49]:
type(d1)

datetime.datetime

In [56]:
# Define a function to convert and reformat the date strings
def parse_and_reformat(date_string):
    # Parse the original date-time string
    datetime_obj = dt.strptime(date_string, "%Y-%m-%dT%H:%M%zZ")
    # Reformat to the new format
    return datetime_obj.strftime(new_format)

In [57]:
# Apply the function to the start_time column
df['start_time'] = df['start_time'].apply(parse_and_reformat)

In [58]:
df.dtypes

start_time          object
energy_generated     int64
dtype: object

In [59]:
df.head()

,start_time,energy_generated
0,2021-12-31 23:45:00,4333
1,2022-01-01 00:00:00,4325
2,2022-01-01 00:15:00,4319
3,2022-01-01 00:30:00,4323
4,2022-01-01 00:45:00,4328


In [22]:
type(d_new)

str

In [23]:
d_final = pd.to_datetime(d_new)

In [60]:
df['start_time'] = pd.to_datetime(df['start_time'])

In [62]:
df.dtypes

start_time          datetime64[ns]
energy_generated             int64
dtype: object

In [61]:
df.head()

,start_time,energy_generated
0,2021-12-31 23:45:00,4333
1,2022-01-01 00:00:00,4325
2,2022-01-01 00:15:00,4319
3,2022-01-01 00:30:00,4323
4,2022-01-01 00:45:00,4328


In [124]:
df['StartTime'] = pd.to_datetime(df['StartTime'])
df['EndTime'] = pd.to_datetime(df['EndTime'])

In [126]:
df.set_index('StartTime', inplace=True)

In [129]:
df.columns

Index(['EndTime', 'AreaID', 'UnitName', 'PsrType', 'quantity',
       'time_interval'],
      dtype='object')

In [130]:
df.drop(['EndTime', 'AreaID', 'UnitName', 'PsrType',
       'time_interval'], inplace=True, axis=1)

In [131]:
hourly_data = df.resample('1H').sum()

In [132]:
hourly_data

,quantity
StartTime,
2021-12-31 23:00:00+00:00,4333
2022-01-01 00:00:00+00:00,17295
2022-01-01 01:00:00+00:00,17322
2022-01-01 02:00:00+00:00,17343
2022-01-01 03:00:00+00:00,17354
...,...
2022-12-31 19:00:00+00:00,18411
2022-12-31 20:00:00+00:00,18058
2022-12-31 21:00:00+00:00,17879


In [115]:
# Removing 'Z' and parsing the StartTime and EndTime columns
df['StartTime'] = df['StartTime'].str[:-1]  # Remove 'Z'
df['EndTime'] = df['EndTime'].str[:-1]  # Remove 'Z'

In [116]:
# Remove '+00:00' and convert to datetime
df['StartTime'] = pd.to_datetime(df['StartTime'].str[:-6])
df['EndTime'] = pd.to_datetime(df['EndTime'].str[:-6])

In [118]:
df.head(10)

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity
0,2022-01-26 23:30:00,2022-01-27 00:00:00,10Y1001A1001A92E,MAW,B19,433
1,2022-01-27 00:00:00,2022-01-27 00:30:00,10Y1001A1001A92E,MAW,B19,472
2,2022-01-27 00:30:00,2022-01-27 01:00:00,10Y1001A1001A92E,MAW,B19,549
3,2022-01-27 01:00:00,2022-01-27 01:30:00,10Y1001A1001A92E,MAW,B19,520
4,2022-01-27 01:30:00,2022-01-27 02:00:00,10Y1001A1001A92E,MAW,B19,407
5,2022-01-27 02:00:00,2022-01-27 02:30:00,10Y1001A1001A92E,MAW,B19,412
6,2022-01-27 02:30:00,2022-01-27 03:00:00,10Y1001A1001A92E,MAW,B19,261
7,2022-01-27 03:00:00,2022-01-27 03:30:00,10Y1001A1001A92E,MAW,B19,165
8,2022-01-27 03:30:00,2022-01-27 04:00:00,10Y1001A1001A92E,MAW,B19,261
9,2022-01-27 04:00:00,2022-01-27 04:30:00,10Y1001A1001A92E,MAW,B19,273


In [98]:
df = df.sort_values(['StartTime']).reset_index(drop=True)

In [113]:
df.head(10)

,quantity
StartTime,
2022-01-26 23:30:00,433
2022-01-26 23:30:00,432
2022-01-27 00:00:00,472
2022-01-27 00:00:00,472
2022-01-27 00:30:00,549
2022-01-27 00:30:00,549
2022-01-27 01:00:00,520
2022-01-27 01:00:00,519
2022-01-27 01:30:00,407


In [105]:
df.drop(['EndTime', 'UnitName', 'AreaID', 'PsrType'], axis=1, inplace=True)

In [109]:
# df.set_index('StartTime', inplace=True)

In [110]:
resampled_df = df.resample('H').sum()

In [112]:
df.head()

,quantity
StartTime,
2022-01-26 23:30:00,433
2022-01-26 23:30:00,432
2022-01-27 00:00:00,472
2022-01-27 00:00:00,472
2022-01-27 00:30:00,549


In [111]:
resampled_df.head()

,quantity
StartTime,
2022-01-26 23:00:00,865
2022-01-27 00:00:00,2042
2022-01-27 01:00:00,1852
2022-01-27 02:00:00,1344
2022-01-27 03:00:00,851


In [83]:
resampled_quantity = df['quantity'].resample('H').sum()


In [84]:
resampled_start_time = df['StartTime'].resample('H').first()  # or use .last()

In [85]:
resampled_df = pd.DataFrame({
    'quantity_sum': resampled_quantity,
    'StartTime_first': resampled_start_time
})

In [86]:
resampled_df

,quantity_sum,StartTime_first
EndTime,,
2022-01-27 00:00:00,1809,2022-01-26 23:30:00
2022-01-27 01:00:00,2137,2022-01-27 00:30:00
2022-01-27 02:00:00,1636,2022-01-27 01:30:00
2022-01-27 03:00:00,851,2022-01-27 02:30:00
2022-01-27 04:00:00,1068,2022-01-27 03:30:00
...,...,...
2022-12-31 19:00:00,838,2022-12-31 18:30:00
2022-12-31 20:00:00,822,2022-12-31 19:30:00
2022-12-31 21:00:00,778,2022-12-31 20:30:00


In [79]:



# Calculating the time interval
df['time_interval'] = df['EndTime'] - df['StartTime']

In [80]:
df.head()

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity,time_interval
0,2022-01-26 23:30:00,2022-01-27 00:00:00,10Y1001A1001A92E,MAW,B19,433,0 days 00:30:00
1,2022-01-27 00:00:00,2022-01-27 00:30:00,10Y1001A1001A92E,MAW,B19,472,0 days 00:30:00
2,2022-01-27 00:30:00,2022-01-27 01:00:00,10Y1001A1001A92E,MAW,B19,549,0 days 00:30:00
3,2022-01-27 01:00:00,2022-01-27 01:30:00,10Y1001A1001A92E,MAW,B19,520,0 days 00:30:00
4,2022-01-27 01:30:00,2022-01-27 02:00:00,10Y1001A1001A92E,MAW,B19,407,0 days 00:30:00


In [18]:
df.drop(['EndTime', 'AreaID', 'UnitName', 'PsrType', 'time_interval'], axis=1, inplace=True)

In [19]:
df.columns

Index(['StartTime', 'quantity'], dtype='object')

In [20]:
df.columns = ['ds', 'y']

In [21]:
df.insert(0, 'unique_id', 1)

In [23]:
df.shape

(12644, 3)

In [25]:
df.head()

,unique_id,ds,y
0,1,2022-01-26 23:30:00+00:00,433
1,1,2022-01-26 23:30:00+00:00,432
2,1,2022-01-27 00:00:00+00:00,472
3,1,2022-01-27 00:00:00+00:00,472
4,1,2022-01-27 00:30:00+00:00,549


In [28]:
df.dtypes

unique_id                  int64
ds           datetime64[ns, UTC]
y                          int64
dtype: object

In [33]:
df.head()

,unique_id,ds,y
0,1,2022-01-26 23:30:00+00:00,433
1,1,2022-01-26 23:30:00+00:00,432
2,1,2022-01-27 00:00:00+00:00,472
3,1,2022-01-27 00:00:00+00:00,472
4,1,2022-01-27 00:30:00+00:00,549


In [29]:
df['ds'] = pd.to_datetime(df['ds'])

In [30]:
df = df.sort_values(['unique_id', 'ds']).reset_index(drop=True)

In [31]:
df.dtypes

unique_id                  int64
ds           datetime64[ns, UTC]
y                          int64
dtype: object

In [32]:
fig = plot_series(df)

ValueError: The time column ('ds') should have either timestamps or integers, got 'object'.

In [1]:
import pandas as pd
import os

def process_files(source_directory, destination_directory):
    # Create the destination directory if it doesn't exist
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    for file in os.listdir(source_directory):
        if file.startswith('gen_') and file.endswith('.csv'):
            file_path = os.path.join(source_directory, file)
            df = pd.read_csv(file_path)

            # Removing 'Z' and parsing the StartTime and EndTime columns
            df['StartTime'] = df['StartTime'].str[:-1]  # Remove 'Z'
            df['EndTime'] = df['EndTime'].str[:-1]  # Remove 'Z'

            df['StartTime'] = pd.to_datetime(df['StartTime'])
            df['EndTime'] = pd.to_datetime(df['EndTime'])

            # Calculating the time interval
            df['time_interval'] = df['EndTime'] - df['StartTime']

            # Saving the modified dataframe in the destination directory
            modified_file_name = file.replace('.csv', '_modified.csv')
            df.to_csv(os.path.join(destination_directory, modified_file_name), index=False)


In [2]:
# Define your source and destination directories
source_directory = '/workspaces/SE-Europe-Data_Challenge/data/raw/2021_to_2022'
destination_directory = '/workspaces/SE-Europe-Data_Challenge/data/processed/2021_to_2022'
process_files(source_directory, destination_directory)

In [3]:
# Define your source and destination directories
source_directory = '/workspaces/SE-Europe-Data_Challenge/data/raw/2022_to_2023'
destination_directory = '/workspaces/SE-Europe-Data_Challenge/data/processed/2022_to_2023'
process_files(source_directory, destination_directory)

In [5]:
# Spain, Sweden, Poland, Italy, Denmark have 1 hour intervals already
# UK has 30 mins intervals, so we group 2 rows
# Netherlands, Germany, Hungary have 15 min intervals, so we group by 4 rows

uk_gen = pd.read_csv("/workspaces/SE-Europe-Data_Challenge/data/processed/2022_to_2023/gen_UK_B19_modified.csv", parse_dates=['StartTime', 'EndTime'])

In [6]:
uk_gen.dtypes

StartTime        datetime64[ns, UTC]
EndTime          datetime64[ns, UTC]
AreaID                        object
UnitName                      object
PsrType                       object
quantity                       int64
time_interval                 object
dtype: object

In [45]:
df = uk_gen.copy(deep=True)

In [46]:
df['Duration'] = df['EndTime'] - df['StartTime']

In [48]:
df.set_index('EndTime', inplace=True)

In [49]:
resampled_quantity = df['quantity'].resample('H').sum()


In [51]:
df.head()

,StartTime,AreaID,UnitName,PsrType,quantity,time_interval,Duration
EndTime,,,,,,,
2022-01-27 00:00:00+00:00,2022-01-26 23:30:00+00:00,10Y1001A1001A92E,MAW,B19,433,0 days 00:30:00,0 days 00:30:00
2022-01-27 00:30:00+00:00,2022-01-27 00:00:00+00:00,10Y1001A1001A92E,MAW,B19,472,0 days 00:30:00,0 days 00:30:00
2022-01-27 01:00:00+00:00,2022-01-27 00:30:00+00:00,10Y1001A1001A92E,MAW,B19,549,0 days 00:30:00,0 days 00:30:00
2022-01-27 01:30:00+00:00,2022-01-27 01:00:00+00:00,10Y1001A1001A92E,MAW,B19,520,0 days 00:30:00,0 days 00:30:00
2022-01-27 02:00:00+00:00,2022-01-27 01:30:00+00:00,10Y1001A1001A92E,MAW,B19,407,0 days 00:30:00,0 days 00:30:00


In [52]:
resampled_quantity.sort_values()

EndTime
2022-07-15 11:00:00+00:00       0
2022-07-01 17:00:00+00:00       0
2022-07-01 16:00:00+00:00       0
2022-07-01 15:00:00+00:00       0
2022-07-01 14:00:00+00:00       0
                             ... 
2022-02-05 10:00:00+00:00    4053
2022-02-05 11:00:00+00:00    4070
2022-02-05 12:00:00+00:00    4073
2022-02-06 12:00:00+00:00    4094
2022-02-05 13:00:00+00:00    4180
Name: quantity, Length: 8136, dtype: int64

In [53]:
resampled_start_time = df['StartTime'].resample('H').first()  # or use .last()


In [54]:
resampled_start_time

EndTime
2022-01-27 00:00:00+00:00   2022-01-26 23:30:00+00:00
2022-01-27 01:00:00+00:00   2022-01-27 00:30:00+00:00
2022-01-27 02:00:00+00:00   2022-01-27 01:30:00+00:00
2022-01-27 03:00:00+00:00   2022-01-27 02:30:00+00:00
2022-01-27 04:00:00+00:00   2022-01-27 03:30:00+00:00
                                       ...           
2022-12-31 19:00:00+00:00   2022-12-31 18:30:00+00:00
2022-12-31 20:00:00+00:00   2022-12-31 19:30:00+00:00
2022-12-31 21:00:00+00:00   2022-12-31 20:30:00+00:00
2022-12-31 22:00:00+00:00   2022-12-31 21:30:00+00:00
2022-12-31 23:00:00+00:00   2022-12-31 22:30:00+00:00
Freq: H, Name: StartTime, Length: 8136, dtype: datetime64[ns, UTC]

In [55]:
resampled_df = pd.DataFrame({
    'quantity_sum': resampled_quantity,
    'StartTime_first': resampled_start_time
})


In [56]:
resampled_df

,quantity_sum,StartTime_first
EndTime,,
2022-01-27 00:00:00+00:00,1809,2022-01-26 23:30:00+00:00
2022-01-27 01:00:00+00:00,2137,2022-01-27 00:30:00+00:00
2022-01-27 02:00:00+00:00,1636,2022-01-27 01:30:00+00:00
2022-01-27 03:00:00+00:00,851,2022-01-27 02:30:00+00:00
2022-01-27 04:00:00+00:00,1068,2022-01-27 03:30:00+00:00
...,...,...
2022-12-31 19:00:00+00:00,838,2022-12-31 18:30:00+00:00
2022-12-31 20:00:00+00:00,822,2022-12-31 19:30:00+00:00
2022-12-31 21:00:00+00:00,778,2022-12-31 20:30:00+00:00


In [40]:
def basic_exploration(file_path):
    df = pd.read_csv(file_path)
    print(f"""\nThe column names in the data are: \n{df.columns}\n{'x' * 60}
          The datatypes of the columns are: \n{df.dtypes}
          {'x' * 60}
          The minimum value of StartTime: {df['StartTime'].min()}
          The maximum value of StartTime: {df['StartTime'].max()}
          The minimum value of EndTime: {df['EndTime'].min()}
          The maximum value of EndTime: {df['EndTime'].max()}
          The minimum value of energy generated:  {df['quantity'].min()}
          The maximum value of energy generated: {df['quantity'].max()}
          """)

In [39]:
basic_exploration("/workspaces/SE-Europe-Data_Challenge/data/processed/2022_to_2023/gen_UK_B19_modified.csv")


The column names in the data are: 
Index(['StartTime', 'EndTime', 'AreaID', 'UnitName', 'PsrType', 'quantity',
       'time_interval'],
      dtype='object')
    xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
          The datatypes of the columns are: 
StartTime        object
EndTime          object
AreaID           object
UnitName         object
PsrType          object
quantity          int64
time_interval    object
dtype: object
          xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
          The minimum value of StartTime: 2022-01-26 23:30:00+00:00
          The maximum value of StartTime: 2022-12-31 23:00:00+00:00
          The minimum value of EndTime: 2022-01-27 00:00:00+00:00
          The maximum value of EndTime: 2022-12-31 23:30:00+00:00
          The minimum value of energy generated:  0
          The maximum value of energy generated: 1052
          
          
          
